<font face="Calibri" size="2"> <i>Open SAR Toolkit - Tutorial 3, version 1.1, November 2019. Andreas Vollrath, ESA/ESRIN phi-lab</i>
</font>

![title](auxiliary/header_image.PNG)

--------

# OST Tutorial III
## Create latest Sentinel-1 GRD product for a given point

--------

**Short description**

This notebook show the interaction between the *Sentinel1* class for data inventory and download, and the *Sentinel1_Scene* class, together, for the generation of the latest Sentinel-1 product over a given point coordinate. 

--------

**Requirements**

- a PC/Mac with at least 16GB of RAM
- about 4GB of free disk space
- a Copernicus Open Data Hub user account, valid for at least 7 days (https://scihub.copernicus.eu)
--------

**NOTE:** all cells that have an * after its number can be executed without changing any code. 

### 1* - Import python libraries necessary for processing

In [ ]:
# this is for the path handling and especially useful if you are on Windows
import os
from os.path import join
from pprint import pprint

# we will need this for our time of period definition
from datetime import datetime, timedelta

# this is the s1Project class, that basically handles all the workflow from beginning to the end
from ost import Sentinel1, Sentinel1_Scene

### 2 - Data selection parameters

**NOTE:** At this point all you want to change is the lat and lon values in line 6.


In order to define the data you want to process you need to define 2 basic settings. 

**1 Area of Interest:** 

In our case we only look for a *specific spot on earth*, that is defined by the *Latitude* and *Longitude*. 

**2 Time of Interest:**

The time of interest is usually defined by a *start* and *end* date. The format should be conform with 'YYYY-MM-DD'. If none of the two parameters are defined, both parameters will use default values, which is the *2014-10-01* for *start*, and *today* for the end of the TOI.

In our example here, the datetime class is used to set the start date to 30 days before today to assure we get any scene within our time of interest.

**3 Project directory**

OST bases usually on a certain directory hierarchy that includes download, inventory and processing. Even though we just want to process a single scene, it is more handy to split these parts for this given example. 



In [ ]:
#----------------------------
# Area of interest
#----------------------------

# Here we can either point to a shapefile or as well use 
lat, lon = 41.8919, 12.5113
aoi = 'POINT ({} {})'.format(lon, lat)

#----------------------------
# Time of interest
#----------------------------
# we set only the start date to today - 30 days
start = (datetime.today() - timedelta(days=30)).strftime("%Y-%m-%d")

#----------------------------
# Processing directory
#----------------------------
# get home folder
home = os.getenv('HOME')
# create a processing directory
project_dir = join(home, 'OpenSarToolkit', 'Tutorial_3')

#------------------------------
# Print out AOI and start date
#------------------------------
print('AOI: ' + aoi, )
print('TOI start: ' + start)

### 3* - Initialize the Sentinel1 project class

In [ ]:
#---------------------------------------------------
# for plotting purposes we use this iPython magic
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (19, 19)
#---------------------------------------------------

# create s1Project class instance
s1_project = Sentinel1(
    project_dir=project_dir, 
    aoi=aoi, 
    start=start,
    product_type='GRD'
)

# search command
s1_project.search()

print('We found {} products.'.format(len(s1_project.inventory.identifier.unique())))
# combine OST class attribute with pandas head command to print out the first 5 rows of the 
print(s1_project.inventory.head(5))

# we plot the full Inventory on a map
s1_project.plot_inventory(transperancy=.1)

### 4* - Select the latest scene found during the search

Here we use some python-panda syntax to filter out the latest scene. If you do not understand what's going on, don't worry. It should just show how 

In [ ]:
pylab.rcParams['figure.figsize'] = (13, 13)

# we give our inventory a shorter name iDf (for inventory Dataframe)
iDf = s1_project.inventory.copy()

# we select the latest scene based on the metadata entry endposition
latestDf = iDf[iDf.endposition == iDf.endposition.max()]

# we print out a little info on the date of the  
print(' INFO: Latest scene found for {}'.format(latestDf['acquisitiondate'].values[0]))

# we use the plotInventory method in combination with the newly
# created Geodataframe to see our scene footprint
s1_project.plot_inventory(latestDf, transperancy=.5)

### 7* Download scene

We use the build-in download method from the *Sentinel1* class. You can pass any Geodataframe generated by OST, and filtered by you (e.g. sort out rows that you d not need). In our case we are only interested in the latest scene, so we pass the newly generated *latestDf* Geodataframe object.'

In [ ]:
s1_project.download(latestDf)

### 8* - Display some metadata of the latest scene

After use of the *Sentinel1* class for the finding and downloading the latest scene, we hand the scene identifier over to the *Sentinel1_Scene* class for further processing. 

**Note** that the *Sentinel1_Scene* class only supports GRD data.

In [ ]:
# create a S1Scene class instance based on the scene identifier coming from our "latest scene dataframe"
latestScene = Sentinel1_Scene(latestDf['identifier'].values[0])

# print summarising infos
latestScene.info()

# print file location
fileLocation = latestScene.get_path(s1_project.download_dir)

print(' File is located at: ')
print(' ' + fileLocation)

### 9* - Produce an ARD product

Our *Sentinel1_Scene* class comes with the build-in method *create_ard* to produce a standardised ARD product. We just have to provide: 
- the high-level download directory (that we can get from our *s1Project* class instance)
- a directory where the outputfiles will be written 
- a filename prefix (the output format will be the standard SNAP Dimap format, consisting of the dim-file and the data directory)
- and a directory for storing temporary files

It seems a bit messy to mix up both classes. In later tutorials it will become clear why this distinction actually makes sense for effective bulk processing.

In [ ]:
latestScene.get_ard_parameters('OST Standard')
latestScene.ard_parameters['single ARD']['resolution'] = 50
pprint(latestScene.ard_parameters)

latestScene.create_ard(infile = latestScene.get_path(download_dir=s1_project.download_dir), # we see our download path can be automatically generated by providing the Project's download directory...wohooo
                       out_dir = s1_project.processing_dir,   
                       out_prefix = latestScene.start_date,   # we take the acquisitiondate as filename for our output
                       temp_dir = s1_project.temp_dir)

print(' The path to our newly created ARD product can be obtained the following way:')
latestScene.ard_dimap


### 10* - Create a RGB color composite

Sentinel-1 scenes usually consist of two polarisation bands. In order to create a 3 channel RGB composite a ratio between the Co- (VV or HH) and the Cross-polarised (VH or HV) band is added. The *create_rgb* method takes the *ard_dimap* file and converts it to a 3-channel GeoTiff.

In [ ]:
latestScene.create_rgb(outfile = opj(s1_project.processing_dir, '{}.tif'.format(latestScene.start_date)))

print(' The path to our newly created RGB product can be obtained the follwing way:')
latestScene.ard_rgb

### 11 - Visualise the RGB composite

We can plot the newly created RGB image with the *visualise_rgb* method. A *shrink_factor* is added, which reduces resolution in favour of memory requirements for plotting.

In [ ]:
latestScene.visualise_rgb(shrink_factor=5)

### 12 - Create thumbnail image

For some it might be interesting to create a small thumbnail image in Jpeg format. The *createThumbnail* method allows for this. 

In [ ]:
tn_image = opj(s1_project.processing_dir, '{}.TN.jpg'.format(latestScene.start_date))
latestScene.create_rgb_thumbnail(outfile = tn_image)

In [ ]:
import imageio
img = imageio.imread(tn_image)
!ls -sh {tn_image}
plt.imshow(img)